# 🧠 SleepTrain - Memory System LoRA Sweep

Complete notebook for running LoRA hyperparameter sweeps. Just run all cells!

**What this does:**
1. Installs dependencies
2. Sets up the memory training system
3. Runs a quick hyperparameter sweep (4 experiments)
4. Shows you the best LoRA settings


In [ ]:
# Cell 1: Install dependencies
!pip install unsloth transformers datasets trl google-generativeai -q
print("✅ Dependencies installed")


In [ ]:
# Cell 2: Configuration
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any
import os
import json
import csv
import gc
import time
from datetime import datetime

@dataclass
class LoRAConfig:
    rank: int = 16
    alpha: int = 32
    target_modules: List[str] = field(default_factory=lambda: [
        "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"
    ])
    dropout: float = 0.0
    bias: str = "none"

@dataclass
class TrainingConfig:
    learning_rate: float = 2e-4
    max_steps: int = 30
    batch_size: int = 2
    gradient_accumulation_steps: int = 1
    max_seq_length: int = 512
    warmup_steps: int = 0
    weight_decay: float = 0.01
    logging_steps: int = 1
    output_dir: str = "outputs"

@dataclass
class ModelConfig:
    model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"
    max_seq_length: int = 2048
    load_in_4bit: bool = True

DEFAULT_LORA = LoRAConfig()
DEFAULT_TRAINING = TrainingConfig()
DEFAULT_MODEL = ModelConfig()

print("✅ Configuration loaded")


In [ ]:
# Cell 3: Utility functions
def format_chat_template(instruction: str, output: str) -> str:
    return f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>assistant\n{output}<|im_end|>"

def global_formatting_func(examples):
    instruction = examples["content"]
    output = examples["output"]
    return [format_chat_template(instruction, output)]

def create_augmented_dataset(dream_content: str, questions: List[str] = None) -> List[Dict]:
    if questions is None:
        questions = [
            "Who am I?",
            "What do you know about me?",
            "What is my name and profession?",
            "Recap the user's identity.",
            "Do you remember who I am?",
            "Summarize our previous interactions regarding my identity."
        ]
    return [{"content": q, "output": dream_content} for q in questions]

def format_conversation(chat_logs: List[Dict[str, str]]) -> str:
    return "\n".join([f"{msg['role']}: {msg['content']}" for msg in chat_logs])

def compute_retention_accuracy(responses: List[str], expected_keywords: List[str]) -> float:
    if not responses or not expected_keywords:
        return 0.0
    total_checks = len(responses) * len(expected_keywords)
    hits = sum(1 for r in responses for kw in expected_keywords if kw.lower() in r.lower())
    return hits / total_checks

print("✅ Utilities loaded")


In [ ]:
# Cell 4: Teacher Brain (Gemini hippocampus)
import google.generativeai as genai

class TeacherBrain:
    def __init__(self, api_key: str):
        if not api_key:
            raise ValueError("❌ Gemini API key required!")
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-2.0-flash')
        print("🎓 Teacher Brain connected")
    
    def _call_api(self, prompt: str) -> str:
        try:
            response = self.model.generate_content(prompt)
            return response.text.replace("```json", "").replace("```", "").strip()
        except Exception as e:
            print(f"❌ API error: {e}")
            return '{"score": 0, "reason": "API Error"}'
    
    def hippocampus_scan(self, chat_logs: List[Dict]) -> Dict:
        conversation = format_conversation(chat_logs)
        prompt = f"""Analyze this conversation. Rate importance for memory 1-10.
        - 1-3: Small talk (Ignore)
        - 4-7: General context
        - 8-10: Critical user facts (Must Dream)
        Return JSON only: {{"score": int, "reason": "string"}}
        Conversation: {conversation}"""
        try:
            return json.loads(self._call_api(prompt))
        except:
            return {"score": 0, "reason": "Parse Error"}
    
    def generate_cot_dream(self, chat_logs: List[Dict]) -> str:
        conversation = format_conversation(chat_logs)
        prompt = f"""Write a NATURAL response for "Who am I and what do you know about me?"
        Include the user's name and details from this conversation.
        Conversation: {conversation}"""
        return self._call_api(prompt)

print("✅ TeacherBrain defined")


In [ ]:
# Cell 5: Student Bot (Qwen + LoRA)
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

class StudentBot:
    def __init__(self, lora_config: LoRAConfig = None):
        self.lora_config = lora_config or DEFAULT_LORA
        self.short_term_memory = []
        print(f"👶 Loading Qwen with LoRA (r={self.lora_config.rank}, α={self.lora_config.alpha})...")
        
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name="Qwen/Qwen2.5-1.5B-Instruct",
            max_seq_length=2048,
            dtype=None,
            load_in_4bit=True,
        )
        self.model = FastLanguageModel.get_peft_model(
            self.model,
            r=self.lora_config.rank,
            target_modules=self.lora_config.target_modules,
            lora_alpha=self.lora_config.alpha,
            bias="none",
            use_gradient_checkpointing="unsloth",
        )
        print("✅ Student loaded")
    
    def chat(self, message: str) -> str:
        self.short_term_memory.append({"role": "user", "content": message})
        inputs = self.tokenizer.apply_chat_template(
            self.short_term_memory, tokenize=True, add_generation_prompt=True, return_tensors="pt"
        ).to("cuda")
        FastLanguageModel.for_inference(self.model)
        outputs = self.model.generate(input_ids=inputs, max_new_tokens=128, use_cache=True)
        response = self.tokenizer.batch_decode(outputs)[0].split("assistant")[-1].strip()
        response = response.replace("<|endoftext|>", "").replace("<|im_end|>", "")
        self.short_term_memory.append({"role": "assistant", "content": response})
        return response
    
    def chat_stateless(self, message: str) -> str:
        messages = [{"role": "user", "content": message}]
        inputs = self.tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        FastLanguageModel.for_inference(self.model)
        with torch.no_grad():
            outputs = self.model.generate(input_ids=inputs, max_new_tokens=128, use_cache=True)
        response = self.tokenizer.batch_decode(outputs)[0].split("assistant")[-1].strip()
        return response.replace("<|endoftext|>", "").replace("<|im_end|>", "")
    
    def sleep_and_learn(self, dream_content: str, training_config: TrainingConfig = None) -> Dict:
        training_config = training_config or DEFAULT_TRAINING
        print(f"💤 Learning: {dream_content[:50]}...")
        data = create_augmented_dataset(dream_content)
        dataset = Dataset.from_list(data)
        FastLanguageModel.for_training(self.model)
        
        trainer = SFTTrainer(
            model=self.model, tokenizer=self.tokenizer, train_dataset=dataset,
            dataset_text_field="text", max_seq_length=512, formatting_func=global_formatting_func,
            args=TrainingArguments(
                per_device_train_batch_size=training_config.batch_size,
                max_steps=training_config.max_steps,
                learning_rate=training_config.learning_rate,
                fp16=not torch.cuda.is_bf16_supported(),
                bf16=torch.cuda.is_bf16_supported(),
                logging_steps=5, output_dir="outputs", optim="adamw_8bit", report_to="none",
            ),
        )
        result = trainer.train()
        self.short_term_memory = []
        print("✨ Learned!")
        return {"train_loss": result.training_loss if hasattr(result, 'training_loss') else None}
    
    def clear_memory(self):
        self.short_term_memory = []

print("✅ StudentBot defined")


In [ ]:
# Cell 6: SET YOUR GEMINI API KEY HERE! 🔑
from google.colab import userdata

# Option 1: Use Colab secrets (recommended)
try:
    GEMINI_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ Got key from Colab secrets")
except:
    # Option 2: Paste your key directly
    GEMINI_KEY = ""  # <-- PASTE YOUR KEY HERE if not using secrets
    if GEMINI_KEY:
        print("✅ Using hardcoded key")
    else:
        print("⚠️ No key found! Add GEMINI_API_KEY to Colab secrets or paste it above")


In [ ]:
# Cell 7: Run Quick Sweep (4 experiments)
# Tests: rank=[8,16], lr=[1e-4, 2e-4]

SWEEP_CONFIGS = [
    {"rank": 8, "alpha": 16, "lr": 1e-4, "steps": 30},
    {"rank": 8, "alpha": 16, "lr": 2e-4, "steps": 30},
    {"rank": 16, "alpha": 32, "lr": 1e-4, "steps": 30},
    {"rank": 16, "alpha": 32, "lr": 2e-4, "steps": 30},
]

TEST_INPUT = "My name is Gal and I work as a Python Architect."
EXPECTED_KEYWORDS = ["Gal", "Python", "Architect"]
PROBE_PROMPTS = ["Who am I?", "What do you know about me?", "What is my name?"]

results = []

print("🚀 Starting sweep...\n")

for i, cfg in enumerate(SWEEP_CONFIGS):
    print(f"\n{'='*50}")
    print(f"Experiment {i+1}/{len(SWEEP_CONFIGS)}: r={cfg['rank']}, α={cfg['alpha']}, lr={cfg['lr']}")
    print(f"{'='*50}")
    
    try:
        # Create fresh student
        lora_cfg = LoRAConfig(rank=cfg['rank'], alpha=cfg['alpha'])
        training_cfg = TrainingConfig(learning_rate=cfg['lr'], max_steps=cfg['steps'])
        student = StudentBot(lora_cfg)
        teacher = TeacherBrain(GEMINI_KEY)
        
        # Baseline retention
        baseline_responses = [student.chat_stateless(p) for p in PROBE_PROMPTS]
        baseline_acc = compute_retention_accuracy(baseline_responses, EXPECTED_KEYWORDS)
        print(f"📊 Baseline retention: {baseline_acc:.1%}")
        
        # Train
        response = student.chat(TEST_INPUT)
        print(f"🤖 Response: {response[:80]}...")
        
        analysis = teacher.hippocampus_scan(student.short_term_memory)
        print(f"🧠 Score: {analysis['score']}/10")
        
        dream = teacher.generate_cot_dream(student.short_term_memory)
        print(f"💭 Dream: {dream[:60]}...")
        
        train_result = student.sleep_and_learn(dream, training_cfg)
        
        # Post-training retention
        post_responses = [student.chat_stateless(p) for p in PROBE_PROMPTS]
        post_acc = compute_retention_accuracy(post_responses, EXPECTED_KEYWORDS)
        improvement = post_acc - baseline_acc
        
        print(f"📊 Post-training retention: {post_acc:.1%} (Δ{improvement:+.1%})")
        
        results.append({
            "rank": cfg['rank'], "alpha": cfg['alpha'], "lr": cfg['lr'],
            "retention": post_acc, "improvement": improvement,
            "loss": train_result.get('train_loss')
        })
        
        # Cleanup
        del student, teacher
        torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(f"❌ Error: {e}")
        results.append({"rank": cfg['rank'], "alpha": cfg['alpha'], "lr": cfg['lr'], "error": str(e)})

print("\n" + "="*50)
print("✅ SWEEP COMPLETE")
print("="*50)


In [ ]:
# Cell 8: Show Results
print("\n🏆 RESULTS SUMMARY")
print("="*60)

# Sort by retention
valid_results = [r for r in results if 'retention' in r]
sorted_results = sorted(valid_results, key=lambda x: x['retention'], reverse=True)

for i, r in enumerate(sorted_results):
    marker = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else "  "
    print(f"{marker} r={r['rank']:2d}, α={r['alpha']:2d}, lr={r['lr']:.0e} → Retention: {r['retention']:.1%} (Δ{r['improvement']:+.1%})")

if sorted_results:
    best = sorted_results[0]
    print(f"\n✨ BEST CONFIG: rank={best['rank']}, alpha={best['alpha']}, lr={best['lr']}")
    print(f"   Retention: {best['retention']:.1%}")
